In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import numpy as np

from tensorflow.keras.optimizers import Adam

import numpy as np

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def preprocess_input_grayscale(img):
    return np.repeat(img, 3, axis=-1)

In [3]:
train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='wrap',
)

train_generator = train_datagen.flow_from_directory(
    'Dataset_train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

val_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input_grayscale,
    rescale=1./255
)

val_generator = val_datagen.flow_from_directory(
    'Dataset_testing',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Found 4444 images belonging to 8 classes.
Found 1114 images belonging to 8 classes.


In [5]:
num_classes = len(train_generator.class_indices)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    tf.keras.layers.Reshape((224, 224 * 3)),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    # tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') #Output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (None, 224, 672)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 224, 128)       │       410,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,184,264 (4.52 MB)

 Trainable params: 1,184,264 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 614ms/step - accuracy: 0.2306 - loss: 2.0085 - precision: 0.0000e+00 - recall: 0.0000e+00

c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 102s 700ms/step - accuracy: 0.2307 - loss: 2.0084 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2531 - val_loss: 1.9716 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/15


c:\Users\rizfi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


139/139 ━━━━━━━━━━━━━━━━━━━━ 54s 391ms/step - accuracy: 0.2435 - loss: 1.9832 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2603 - val_loss: 1.9457 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 56s 399ms/step - accuracy: 0.2511 - loss: 1.9662 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2558 - val_loss: 1.9403 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 216us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 73s 523ms/step - accuracy: 0.2683 - loss: 1.9588 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2406 - val_loss: 1.9669 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 8/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s

In [7]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = history.history['accuracy']  # Akurasi pada data training
val_accuracy = history.history['val_accuracy']  # Akurasi pada data validasi

# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2459
Epoch 2: 0.0000
Epoch 3: 0.2532
Epoch 4: 0.0000
Epoch 5: 0.2550
Epoch 6: 0.0000
Epoch 7: 0.2556
Epoch 8: 0.0000
Epoch 9: 0.2489
Epoch 10: 0.0000
Epoch 11: 0.2518
Epoch 12: 0.0000
Epoch 13: 0.2518
Epoch 14: 0.0000
Epoch 15: 0.2574

Validation Accuracy per Epoch:
Epoch 1: 0.2531
Epoch 2: 0.2603
Epoch 3: 0.2558
Epoch 4: 0.2406
Epoch 5: 0.2558
Epoch 6: 0.2585
Epoch 7: 0.2513
Epoch 8: 0.2585


In [8]:

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall'])

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    verbose=1
)

Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 64s 428ms/step - accuracy: 0.2851 - loss: 1.9254 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2585 - val_loss: 1.9390 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 54s 384ms/step - accuracy: 0.2632 - loss: 1.9379 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2621 - val_loss: 1.9385 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 214us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 55s 392ms/step - accuracy: 0.2657 - loss: 1.9410 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2630 - val_loss: 1.9384 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/20
139/139 ━━━━━━━━━━━━

In [9]:
# Mendapatkan nilai akurasi untuk setiap epoch dari history
train_accuracy = fine_tune_history.history['accuracy']  # Akurasi pada data training
val_accuracy = fine_tune_history.history['val_accuracy']  # Akurasi pada data validasi
# Mencetak akurasi di setiap epoch
print("Training Accuracy per Epoch:")
for epoch, acc in enumerate(train_accuracy, 1):
    print(f"Epoch {epoch}: {acc:.4f}")

print("\nValidation Accuracy per Epoch:")
for epoch, val_acc in enumerate(val_accuracy, 1):
    print(f"Epoch {epoch}: {val_acc:.4f}")

Training Accuracy per Epoch:
Epoch 1: 0.2590
Epoch 2: 0.0000
Epoch 3: 0.2637
Epoch 4: 0.0000
Epoch 5: 0.2640
Epoch 6: 0.0000
Epoch 7: 0.2622
Epoch 8: 0.0000
Epoch 9: 0.2581
Epoch 10: 0.0000
Epoch 11: 0.2658
Epoch 12: 0.0000
Epoch 13: 0.2572
Epoch 14: 0.0000
Epoch 15: 0.2595
Epoch 16: 0.0000
Epoch 17: 0.2586
Epoch 18: 0.0000
Epoch 19: 0.2597
Epoch 20: 0.0000

Validation Accuracy per Epoch:
Epoch 1: 0.2585
Epoch 2: 0.2621
Epoch 3: 0.2630
Epoch 4: 0.2621
Epoch 5: 0.2621
Epoch 6: 0.2603
Epoch 7: 0.2603
Epoch 8: 0.2612
Epoch 9: 0.2612
Epoch 10: 0.2621


In [11]:
test_images, test_labels = next(val_generator)

test_images = test_images[:10]
test_labels = test_labels[:10]

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Tampilkan hasil prediksi untuk 10 data
for i in range(10):
    print(f"Predicted: {predicted_classes[i]}, True: {true_classes[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted: 4, True: 7
Predicted: 4, True: 4
Predicted: 4, True: 0
Predicted: 4, True: 2
Predicted: 4, True: 7
Predicted: 4, True: 4
Predicted: 4, True: 7
Predicted: 4, True: 4
Predicted: 4, True: 3
Predicted: 4, True: 4


In [ ]:
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels from the test generator
y_true = val_generator.classes  # True class labels

# Get class names from the generator
class_names = list(val_generator.class_indices.keys())

# Generate the classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# Optional: Confusion matrix for additional insights
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)